# G-League: Webscraping

### Brian Pang

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import os
import unidecode

#### Player Statistics - website: www.basketball-reference.com

Loop that scrapes the past 10 years of per game G-league player statistics and creates a dictionary of all seasons data. I also use this loop to scrape the past 10 years of NBA statistics to extract the names of players who played in the NBA for each season.

In [2]:
G_league_dict = {}
full_NBA_Players_df = pd.DataFrame([])

year = range(2010,2020)
for i in range(len(year)):
    url = 'https://www.basketball-reference.com/gleague/years/gleague_{}_per_game.html'.format(year[i])
    html = requests.get(url)
    soup = BeautifulSoup(html.content)
    

    headers = [th.getText() for th in soup.findAll('tr')[0].findAll('th')]
    headers
    
    
    
    rows = soup.findAll('tr')[0:]
    player_names = [[th.getText() for th in rows[i].findAll('th')] for i in range(len(rows))]
    player_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]
    

    del player_names[::11]
    del player_stats[::11]

    stats = pd.DataFrame(player_stats, columns = headers[1:])
    
    stats['Player'] = [item for sublist in player_names for item in sublist]
    stats['Season'] = year[i]
    
    G_league_dict['gleaguestats{}'.format(str(2010+i))] = stats
    
    #NBA Statistics
    nba_url = 'https://www.basketball-reference.com/leagues/NBA_{}_per_game.html'.format(year[i])
    nba_html = requests.get(nba_url)
    soup = BeautifulSoup(nba_html.content)
    
    headers = [th.getText() for th in soup.findAll('tr')[0].findAll('th')]
    rows = soup.findAll('tr')[0:]
    nba_players = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]
    nba_players_df = pd.DataFrame(nba_players, columns = headers[1:])
    nba_players_df['Season'] = year[i]
    full_NBA_Players_df = full_NBA_Players_df.append(nba_players_df)

In [3]:
G_league_dict['gleaguestats2010']

,Tm,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player,Season
0,TUL,,9,2,22.4,2.2,5.6,.400,0.1,1.1,...,2.8,3.1,3.0,1.7,0.0,1.6,1.8,4.7,Wink Adams,2010
1,MAI,25,22,19,34.4,5.3,11.2,.476,1.5,3.5,...,3.9,4.5,3.4,0.9,0.2,2.7,2.0,15.6,Maurice Ager,2010
2,ABQ,25,50,43,32.1,5.2,11.2,.463,1.9,4.4,...,2.1,2.5,2.8,1.0,0.0,1.9,1.5,15.1,Antoine Agudio,2010
3,TOT,25,30,23,38.1,5.6,13.9,.401,2.5,7.0,...,2.6,3.1,5.2,1.2,0.2,3.1,2.5,19.2,Blake Ahearn,2010
4,BAK,25,17,10,32.8,4.3,11.7,.367,1.6,5.9,...,1.6,2.1,4.1,1.2,0.1,2.5,2.4,14.3,Blake Ahearn,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,DAK,,50,29,24.5,4.0,8.1,.496,0.0,0.1,...,4.4,6.7,0.9,1.2,0.8,2.5,3.1,11.3,Curtis Withers,2010
359,LOS,,43,19,28.7,2.3,5.4,.416,0.6,1.8,...,2.5,2.8,3.8,1.0,0.0,1.8,2.4,6.9,Horace Wormely,2010
360,BAK,,2,0,14.0,1.0,5.5,.182,0.0,0.5,...,2.0,2.5,0.0,0.0,0.0,1.5,0.5,4.0,Akeem Wright,2010
361,LOS,,30,23,27.1,5.6,11.0,.508,0.6,1.5,...,1.2,1.5,3.0,0.5,0.1,1.5,0.9,13.4,James Wright,2010


In [4]:
full_NBA_Players_df.head(10)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Season
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,2010
1,Arron Afflalo,SG,24,DEN,82,75,27.1,3.3,7.1,.465,...,0.7,2.4,3.1,1.7,0.6,0.4,0.9,2.7,8.8,2010
2,Alexis Ajinça,C,21,CHA,6,0,5.0,0.8,1.7,.500,...,0.2,0.5,0.7,0.0,0.2,0.2,0.3,0.8,1.7,2010
3,LaMarcus Aldridge,PF,24,POR,78,78,37.5,7.4,15.0,.495,...,2.5,5.6,8.0,2.1,0.9,0.6,1.3,3.0,17.9,2010
4,Joe Alexander,SF,23,CHI,8,0,3.6,0.1,0.8,.167,...,0.3,0.4,0.6,0.3,0.1,0.1,0.0,1.1,0.5,2010
5,Malik Allen,PF,31,DEN,51,3,8.9,0.9,2.3,.397,...,0.7,0.9,1.6,0.3,0.2,0.1,0.4,1.3,2.1,2010
6,Ray Allen*,SG,34,BOS,80,80,35.2,5.8,12.2,.477,...,0.6,2.6,3.2,2.6,0.8,0.3,1.6,2.3,16.3,2010
7,Tony Allen,SG,28,BOS,54,8,16.5,2.4,4.7,.510,...,1.0,1.7,2.7,1.3,1.1,0.4,1.2,2.0,6.1,2010
8,Rafer Alston,PG,33,TOT,52,38,27.3,3.0,8.6,.348,...,0.3,2.2,2.5,3.4,1.0,0.2,1.8,2.0,8.2,2010
9,Rafer Alston,PG,33,NJN,27,13,28.4,3.5,10.3,.343,...,0.3,2.4,2.8,3.9,1.0,0.2,2.3,1.9,9.7,2010


Clean up the full_NBA_Players_df dataframe:
1. Remove rows with None (blank spaces) in more than 10 columns for all rows
2. Fix the player names so that any accents are removed - so that it can be matched with g-league stats
3. Remove the * next to player names (which indicate if a player is in the Hall of Fame)



In [5]:
full_NBA_Players_df.dropna(thresh = 10, inplace = True)
full_NBA_Players_df.Player = full_NBA_Players_df.Player.apply(unidecode.unidecode)
full_NBA_Players_df.Player = full_NBA_Players_df.Player.str.replace(r'*', '')
full_NBA_Players_df.head(10)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Season
1,Arron Afflalo,SG,24,DEN,82,75,27.1,3.3,7.1,.465,...,0.7,2.4,3.1,1.7,0.6,0.4,0.9,2.7,8.8,2010
2,Alexis Ajinca,C,21,CHA,6,0,5.0,0.8,1.7,.500,...,0.2,0.5,0.7,0.0,0.2,0.2,0.3,0.8,1.7,2010
3,LaMarcus Aldridge,PF,24,POR,78,78,37.5,7.4,15.0,.495,...,2.5,5.6,8.0,2.1,0.9,0.6,1.3,3.0,17.9,2010
4,Joe Alexander,SF,23,CHI,8,0,3.6,0.1,0.8,.167,...,0.3,0.4,0.6,0.3,0.1,0.1,0.0,1.1,0.5,2010
5,Malik Allen,PF,31,DEN,51,3,8.9,0.9,2.3,.397,...,0.7,0.9,1.6,0.3,0.2,0.1,0.4,1.3,2.1,2010
6,Ray Allen,SG,34,BOS,80,80,35.2,5.8,12.2,.477,...,0.6,2.6,3.2,2.6,0.8,0.3,1.6,2.3,16.3,2010
7,Tony Allen,SG,28,BOS,54,8,16.5,2.4,4.7,.510,...,1.0,1.7,2.7,1.3,1.1,0.4,1.2,2.0,6.1,2010
8,Rafer Alston,PG,33,TOT,52,38,27.3,3.0,8.6,.348,...,0.3,2.2,2.5,3.4,1.0,0.2,1.8,2.0,8.2,2010
9,Rafer Alston,PG,33,NJN,27,13,28.4,3.5,10.3,.343,...,0.3,2.4,2.8,3.9,1.0,0.2,2.3,1.9,9.7,2010
10,Rafer Alston,PG,33,MIA,25,25,26.2,2.4,6.8,.355,...,0.2,2.0,2.2,2.9,0.9,0.2,1.4,2.1,6.6,2010


#### Callups: Scrape G-league site for all callups for all g-league seasons. Website: https://gleague.nba.com/all-time-callups/

In [6]:
URL = 'https://gleague.nba.com/all-time-callups/'
r = requests.get(URL)

soup = BeautifulSoup(r.content)


G_callups = [soup.findAll('td')[i].getText() for i in range(len(soup.findAll('td')))]

G_callups_df = pd.DataFrame(G_callups, columns = ['Data'])


tf = G_callups_df['Data'].str.contains('SEASON')

#Rows that contain season - indicates a new season
np.where(tf)

#Split the G_callups_df into separate datasets based on which year the callup occurred
callups_2018 = G_callups_df[0:307].reset_index(drop=True)[7::5]
callups_2017 = G_callups_df[307:569].reset_index(drop=True)[7::5]
callups_2016 = G_callups_df[569:786].reset_index(drop=True)[7::5]
callups_2015 = G_callups_df[786:1113].reset_index(drop=True)[7::5]
callups_2014 = G_callups_df[1113:1365].reset_index(drop=True)[2::5]
callups_2013 = G_callups_df[1365:1551].reset_index(drop=True)[2::5]
callups_2012 = G_callups_df[1551:1734].reset_index(drop=True)[2::3]
callups_2011 = G_callups_df[1734:1820].reset_index(drop=True)[4::3]
callups_2010 = G_callups_df[1820:1945].reset_index(drop=True)[4::3]


#Callups from the 2018-2019 are on another separate webpage
soup = BeautifulSoup(requests.get('https://gleague.nba.com/callups/?season=2018').content)
rows = soup.findAll('a')[0:]
callups_2019 = pd.DataFrame([soup.findAll('a',attrs = {'class':'table-list__player-name'})[i].getText() for i in range(len(soup.findAll('a',attrs = {'class':'table-list__player-name'})))])


list_of_callups = [callups_2019, callups_2018,callups_2017,callups_2016,callups_2015,callups_2014,callups_2013,callups_2012,callups_2011,callups_2010]    
year = 2019

#Now combine all the callups datasets into one - Callups_df
Callups_df = pd.DataFrame([],columns = ['Player','Season'])
year = 2019
for dataset in list_of_callups:
    dataset['Season'] = year
    dataset.columns = ['Player', 'Season']
    Callups_df = Callups_df.append(dataset)
    year-=1

In [7]:
Callups_df.head(5)

,Player,Season
0,Kendrick Nunn,2019
1,Tarik Phillip,2019
2,Michael Frazier,2019
3,Billy Garrett,2019
4,BJ Johnson,2019


#### Assignments: Repeat Procedure of scraping callups for assignments. Website: https://gleague.nba.com/all-time-nba-assignments/

In [8]:
soup = BeautifulSoup(requests.get('https://gleague.nba.com/all-time-nba-assignments/').content)
assignments = [soup.findAll('td')[i].getText() for i in range(len(soup.findAll('td')))]


G_league_assignments_df = pd.DataFrame(assignments, columns = ['Data'])
tempname1 = G_league_assignments_df['Data'].str.contains('SEASON|Season')
np.where(tempname1)

assignments_2018 = G_league_assignments_df[0:2182].reset_index(drop=True)[7::5]
assignments_2017 = G_league_assignments_df[2182:4539].reset_index(drop=True)[7::5]
assignments_2016 = G_league_assignments_df[4539:6146].reset_index(drop=True)[7::5]
assignments_2015 = G_league_assignments_df[6146:7119].reset_index(drop=True)[7::5]
assignments_2014 = G_league_assignments_df[7119:8237].reset_index(drop=True)[2::6]
assignments_2013 = G_league_assignments_df[8237:9331].reset_index(drop=True)[2::6]
assignments_2012 = G_league_assignments_df[9331:9524].reset_index(drop=True)[4::3]
assignments_2011 = G_league_assignments_df[9524:9645].reset_index(drop=True)[4::3]
assignments_2010 = G_league_assignments_df[9645:9723].reset_index(drop=True)[3::3]


#G league assignments from 2018-2019 season were on a separate webpage
soup = BeautifulSoup(requests.get('https://gleague.nba.com/assignments/?season=2018').content)
rows=soup.findAll('a')[0:]
assignments_2019 = pd.DataFrame([soup.findAll('a', attrs = {'class':'table-list__player-name'})[i].getText() for i in range(len(soup.findAll('a', attrs = {'class':'table-list__player-name'})))])

list_of_assignments = [assignments_2019,assignments_2018,assignments_2017,assignments_2016,assignments_2015,assignments_2014,assignments_2013,assignments_2012,assignments_2011,assignments_2010]

Assignments_df = pd.DataFrame([],columns = ['Player', 'Season'])
year = 2019
for dataset in list_of_assignments:
    dataset['Season'] = year
    dataset.columns = ['Player','Season']
    Assignments_df = Assignments_df.append(dataset)
    year -=1

In [9]:
Assignments_df.head(5)

,Player,Season
0,Dzanan Musa,2019
1,Theo Pinson,2019
2,Michael Frazier,2019
3,Gary Clark,2019
4,Dzanan Musa,2019


#### Two-Way Players - website: www.hoopsrumors.com

In [10]:
def get_two_way_players(link, year):
    soup = BeautifulSoup(requests.get(link).content)
    dataframe = pd.DataFrame([soup.findAll('a',  attrs = {'target': '_blank', 'rel': 'nofollow noopener'})[i].getText() for i in range(len(soup.findAll('a',attrs = {'target': '_blank', 'rel': 'nofollow noopener'})))])
    dataframe = dataframe.rename({0:'Player'}, axis = 1)
    dataframe['Year'] = year
    return dataframe


two_way_2019 = get_two_way_players('https://www.hoopsrumors.com/2018/07/201819-nba-two-way-contract-tracker.html', 2019)
two_way_2018 = get_two_way_players('https://www.hoopsrumors.com/2017/07/201718-nba-two-way-contract-tracker.html', 2018)

two_ways = two_way_2019.append(two_way_2018)

In [11]:
two_ways.head(5)

,Player,Year
0,Quinn Cook,2019
1,Tyrone Wallace,2019
2,Alex Poythress,2019
3,R.J. Hunter,2019
4,Alan Williams,2019


In [12]:
#Loop to create the combined dataframe of the player statistcs, and new columns for callups, assignments, two-way players
G_league_stats_df = pd.DataFrame([])
year = 2010
for key, df in G_league_dict.items():
    df['NBA Minutes'] = df.Player.isin(full_NBA_Players_df[full_NBA_Players_df['Season'] == year].Player)
    df['Called Up'] = df.Player.isin(Callups_df[Callups_df['Season'] == year].Player)
    df['Assigned'] = df.Player.isin(Assignments_df[Assignments_df['Season'] == year].Player)
    df['Two-Way Player'] = df.Player.isin(two_ways[two_ways['Year'] == year].Player)
    G_league_stats_df = G_league_stats_df.append(df)
    year += 1


#Convert boolean variables to numeric
Bool_to_numeric = {True: 1, False: 0}

for i in ['NBA Minutes', 'Called Up', 'Assigned', 'Two-Way Player']:
    G_league_stats_df[i].replace(Bool_to_numeric,inplace =True)


In [13]:
G_league_stats_df.head(5)

,Tm,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,...,BLK,TOV,PF,PTS,Player,Season,NBA Minutes,Called Up,Assigned,Two-Way Player
0,TUL,,9,2,22.4,2.2,5.6,.400,0.1,1.1,...,0.0,1.6,1.8,4.7,Wink Adams,2010,0,0,0,0
1,MAI,25,22,19,34.4,5.3,11.2,.476,1.5,3.5,...,0.2,2.7,2.0,15.6,Maurice Ager,2010,0,0,0,0
2,ABQ,25,50,43,32.1,5.2,11.2,.463,1.9,4.4,...,0.0,1.9,1.5,15.1,Antoine Agudio,2010,0,0,0,0
3,TOT,25,30,23,38.1,5.6,13.9,.401,2.5,7.0,...,0.2,3.1,2.5,19.2,Blake Ahearn,2010,0,0,0,0
4,BAK,25,17,10,32.8,4.3,11.7,.367,1.6,5.9,...,0.1,2.5,2.4,14.3,Blake Ahearn,2010,0,0,0,0


In [14]:
#This loop removes all rows of a player's stats in a season beyond the TOT row, if that player has multiple entries for a season
G_df_rm_duplicates = pd.DataFrame([])
years = G_league_stats_df['Season'].unique()
for each in years:
    G_df_rm_duplicates = G_df_rm_duplicates.append(G_league_stats_df.loc[G_league_stats_df.Season==each].drop_duplicates(subset = ['Player']).copy())


In [15]:
G_df_rm_duplicates.head(5)

,Tm,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,...,BLK,TOV,PF,PTS,Player,Season,NBA Minutes,Called Up,Assigned,Two-Way Player
0,TUL,,9,2,22.4,2.2,5.6,.400,0.1,1.1,...,0.0,1.6,1.8,4.7,Wink Adams,2010,0,0,0,0
1,MAI,25,22,19,34.4,5.3,11.2,.476,1.5,3.5,...,0.2,2.7,2.0,15.6,Maurice Ager,2010,0,0,0,0
2,ABQ,25,50,43,32.1,5.2,11.2,.463,1.9,4.4,...,0.0,1.9,1.5,15.1,Antoine Agudio,2010,0,0,0,0
3,TOT,25,30,23,38.1,5.6,13.9,.401,2.5,7.0,...,0.2,3.1,2.5,19.2,Blake Ahearn,2010,0,0,0,0
6,MAI,21,22,21,26.0,5.1,9.4,.546,0.0,0.1,...,3.1,3.0,3.9,14.6,Alexis Ajinca,2010,1,0,1,0


In [16]:
G_df_rm_duplicates.to_csv("C://Users/bpang/Downloads/gleaguestatsdata.csv", index = False)